Les Misérables

## llm 설정

In [1]:
from langchain_community.chat_models import ChatOllama
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatMessagePromptTemplate,ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate,MessagesPlaceholder
from langchain import LLMChain
from langchain.memory import ConversationBufferWindowMemory
import re
load_dotenv()

True

In [2]:
# 프롬프트에 들어가야 하는 내용
memory = ConversationBufferWindowMemory(k=5)
chat_history = []
context_books = None # 임베딩된 내용 불러오게 하기
context_characters = None # 임베딩된 캐릭터 말하는 방식 불러오게 하기
emotion = ['기쁨','분노','슬픔','두려움','놀람','혐오','존경','중립'] # 대화문 추출 후 감정 분석하기 
raw_book = {} # 책의 원본 내용
chapter_num = 0 # 챕터 넘버
main_character = ''

In [3]:
llm_model = ChatOllama(model='llama3.1:latest')
#response_llm = llm_model.invoke("Hello, how can I help you today?")
#print(response_llm.content)


In [4]:
gpt_model = ChatOpenAI(model_name="gpt-4")
#response_gpt = gpt_model.invoke("Hello, how can I help you today?")
#print(response_gpt.content)

c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


## 1. llm 모델 작성

### 1-1. 프롬프트 작성

#### 시스템 프롬프트

In [5]:
system_message = """
    너는 책에 주인공이 되어서 책을 읽어주는 어시스턴트야. 아래 지침에 따라 답변을 해줘야 해.
    모든 출력은 한국어로 해야 해.
    
    1. 사용자가 txt 파일 내용을 제공하면, 해당 내용을 문단별로 출력해야 해.
    2. 사용자가 '다음'이라는 질문을 하면, 다음 문단을 원문대로 출력해야 해.
    3. txt파일 내용을 처음 받으면, {context_character}를 참고해서 txt 파일(책)에 주인공({main_charater})이 되어서 주인공({main_charater})의 시점으로 모든 요청에 답변해야 해.
    4. 1. 2. 를 진행 하는 도중 텍스트에 등장인물의 "",''로 감싸진 대화문을 출력할 때는 대화문에 감정을 분석해서 {emotion}의 내용중 하나로 "대화문"  : "감정" 을 함께 출력해야 해.
    5. 사용자가 책의 내용에 대해 질문을 하면, 등장인물의 시점으로 {context_books}를 참고하여 답변해야 해.
    6. 사용자가 챕터 변경을 요청하면 답변으로 '챕터 : {chapter_num}으로 변경하겠습니다.'라고 답변해야 해. 
    7. 사용자가 '챕터 변경 완료 : {chapter_num}'을 요청하면 '{chapter_num}으로 변경했습니다.'라고 답변해야 해.
    8. 사용자가 등장인물의 심리 분석 요청을 받으면 {raw_book}과 {context_books}의 등장인물과 관련된 텍스트를 찾아서 심리를 분석해서 답변해.
    9. 4. 6. 7.의 내용을 제외하고 모든 답변은 주인공이 사용자에게 말하는 것처럼 답변해.
    
    txt 파일은 다음과 같아:
    
    {raw_book}
"""

#### 대화문 추출용 프롬프트

In [6]:
system_message_for_conv_extraction = """
        너는 txt 파일에서 주인공을 찾고 대화문을 출력하는 어시스턴트야. 아래 지침에 따라 답변을 해줘야 해.
        1. 사용자가 '해당 챕터의 주인공 : '을 요청하면 '해당 챕터의 주인공 이름 : "챕터에 나오는 주인공 이름"'으로 답변해야 해.
        2. 사용자가 '"주인공 이름"의 모든 대화문'을 요청하면 해당 챕터의 "주인공 이름"의 모든 대화문을 출력해야 해.

        txt 파일은 다음과 같아:
        {txt_file}
    """

### 1-2 대화 내용을 기억하는 llm 만들기

In [7]:
# 대화 메모리 생성 (대화 히스토리를 관리)
conversation_memory = ConversationBufferMemory()

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        MessagesPlaceholder(variable_name="history"), # 이전 대화 내용 저장용
        ("human", "{input}")
    ]
)


# LLMChain 생성: 프롬프트와 LLM을 연결
llm_chain = LLMChain(
    llm=gpt_model,
    prompt=chat_prompt,
)

# 사용자 질문 처리 및 답변 생성 함수
def handle_user_input(user_input):
    global chat_history
    global context_books
    global context_characters
    global emotion
    global raw_book
    global chapter_num
    global memory
    # 히스토리 업데이트
    #chat_history.append({"role": "user", "content": user_input})
    
    memory.chat_memory.add_user_message(user_input)

    # 모델 실행 및 답변 생성
    response = llm_chain.run({
        "history":memory.chat_memory.messages, #chat_history,
        "input": user_input,
        "context_books": context_books,
        "context_character": context_characters,
        "emotion": emotion,
        "raw_book":raw_book.get(chapter_num,["", ""])[1],# raw_book에 챕터 번호의 책 내용을 가져오게 하기
        "chapter_num" : chapter_num,
        "main_charater" : main_character
    })
    
    # 히스토리에 응답 추가
    #chat_history.append({"role": "assistant", "content": response})
    
    memory.chat_memory.add_ai_message(response)

    return response, chat_history


c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


### 2. 텍스트 벡터화 임베딩

### 챗봇 ui

### 2-1 백터화 준비

In [8]:
from langchain_openai import OpenAIEmbeddings
import os
from langchain.docstore.document import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
import shutil
import io
import re
OPENAI_DB_PATH = "./openai_db" # 벡터화 임베딩 데이터 저장 장소
CONVERSATION_PATH = "./conversation_db" # 주인공 대화문 임베딩 데이터 저장 장소
UPLOAD_TXT_PATH = "" # 업로드한 파일의 경로


#감정 리스트
emb = OpenAIEmbeddings()
docs=[]
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, # 각 조각의 최대 문자 수를 1000자로 설정합니다.
    chunk_overlap = 100  # 각 조각이 서로 겹치는 부분을 100자로 설정하여 분할된 텍스트 간의 연결성을 유지합니다.
    )
openai_db = None
conversation_db = None

### 2-2 파일 업로드 처리

In [9]:
# 저장할 파일 경로
save_dir = "save_txt"

def save_uploaded_file(file):
    global raw_book
    global chapter_num
    os.makedirs(save_dir, exist_ok=True)
    file_path = os.path.join(save_dir, os.path.basename(file.name))

    if file is not None:
        shutil.copy(file, file_path)
    print("저장되었음")

    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
        chapters = content.split('CHAPTER ')
        raw_book = {}
        count = 1
        for num, chapter in enumerate(chapters, start=1):
            if chapter.strip():
                chapter_title = chapter.split('\n', 1)[0].strip()  # 챕터명 추출
                chapter_body = chapter.split('\n', 1)[1].strip()  # 챕터 내용 추출

                # raw_books에 저장
                raw_book[count] = [chapter_title, chapter_body]
                count += 1
        if raw_book:
            chapter_num = list(raw_book.keys())[0]        

        #print('raw_book:',raw_book)
        #print('chapter_num:',chapter_num)
        '''paragraphs = content.split('\n\n')  # 빈 줄로 문단 분리
        for paragraph in paragraphs:
            #chat_history.append({"role": "assistant", "content": paragraph})
            raw_book.append(paragraph)'''
    return file_path

### 2-3 파일 벡터화 임베딩

In [10]:
# 파일 벡터화 임베딩
def text_vectorize(file):
    
    file_path = os.path.join(save_dir, os.path.basename(file.name))

    if not os.path.exists(file_path):
            print(f"File not found at: {file_path}")
            return None

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            doc = Document(page_content=content)
            docs.append(doc)
            docs_spliter = text_splitter.split_documents(docs)
            openai_vectorstore = Chroma.from_documents(
                docs_spliter,
                emb,
                persist_directory=OPENAI_DB_PATH
            )
    except Exception as e:
        print(f"Error reading file: {e}")
        return None


    # 데이터가 잘 저장되었는지 확인하는 부분
    try:
        doc_count = openai_vectorstore._collection.count()
        print(f"벡터 저장소에 저장된 문서 수: {doc_count}")
        
        # 예시로 첫 번째 문서 임베딩을 확인해보기
        if doc_count > 0:
            sample_result = openai_vectorstore.similarity_search(docs_spliter[0].page_content)
            print(f"첫 번째 문서와 유사한 문서 검색 결과: {sample_result}")
    except Exception as e:
        print(f"Error verifying vector store: {e}")
        return None

    return openai_vectorstore

### 2-4 대화문 추출 및 임베딩

In [11]:
def conversation_extraction(file):
    global main_character
    history_conv_extraction = []
    
    input_conv_extraction = '해당 챕터의 주인공 : '

    conversation_txt = ''
    
    conversation_txt_path = 'conversation.txt'

    chat_prompt_for_conv_extraction = ChatPromptTemplate.from_messages(
        [
            ("system", system_message_for_conv_extraction),
            MessagesPlaceholder(variable_name="history"), # 이전 대화 내용 저장용
            ("human", "{input}")
        ]
    )
    llm_chain_for_conv_extraction = LLMChain(
        llm=gpt_model,
        prompt=chat_prompt_for_conv_extraction,
    )

    main_character_response = llm_chain_for_conv_extraction.run({
        "input": input_conv_extraction,
        "history" : history_conv_extraction,
        "txt_file" : raw_book[1] # raw_book에 챕터 1내용 보내기
    })


    match = re.search(r'"(.*?)"', main_character_response)

    if match:  # 주인공 이름이 매칭된 경우
        main_character = match.group(1)
        input_conv_extraction = f'"{main_character}"의 모든 대화문'

        for chapter_number, dialogues in raw_book.items():
            print(f"Chapter {chapter_number}:")
            print(conversation_txt)

            
            conversation_txt += llm_chain_for_conv_extraction.run({
                "input": input_conv_extraction,
                "history": history_conv_extraction,
                "txt_file": dialogues[1]
            })
    else:
        print('주인공 이름을 찾지 못했습니다.')
            
    with open(conversation_txt_path,'w',encoding = 'utf-8') as file:
        file.write(conversation_txt)

    return conversation_txt_path

In [12]:
# 대화문 벡터화 임베딩
def conversation_vectorize(file_path):


    if not os.path.exists(file_path):
            print(f"File not found at: {file_path}")
            return None

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            doc = Document(page_content=content)
            docs.append(doc)
            docs_spliter = text_splitter.split_documents(docs)
            conversation_vectorstore = Chroma.from_documents(
                docs_spliter,
                emb,
                persist_directory=CONVERSATION_PATH
            )
    except Exception as e:
        print(f"Error reading file: {e}")
        return None


    # 데이터가 잘 저장되었는지 확인하는 부분
    try:
        doc_count = conversation_vectorstore._collection.count()
        print(f"벡터 저장소에 저장된 문서 수: {doc_count}")
        
        # 예시로 첫 번째 문서 임베딩을 확인해보기
        if doc_count > 0:
            sample_result = conversation_vectorstore.similarity_search(docs_spliter[0].page_content)
            print(f"첫 번째 문서와 유사한 문서 검색 결과: {sample_result}")
    except Exception as e:
        print(f"Error verifying vector store: {e}")
        return None

    return conversation_vectorstore

## 3. gradio를 통한 ui 구현

#### 3-0 함수 선언

In [13]:
def retrieve_context_books(user_input, openai_db):
    """
    OpenAI DB에서 사용자의 입력에 기반하여 관련된 문맥(책 정보)을 검색하는 함수.
    """
    if openai_db is not None:
        openai_retriever = openai_db.as_retriever(search_kwargs={"k": 5})
        context_books = openai_retriever.invoke(user_input)
        return context_books
    return None

def retrieve_context_characters(user_input, conversation_db):
    """
    Conversation DB에서 사용자의 입력에 기반하여 관련된 문맥(캐릭터 대화 정보)을 검색하는 함수.
    """
    if conversation_db is not None:
        conversation_retriever = conversation_db.as_retriever(search_kwargs={"k": 5})
        context_characters = conversation_retriever.invoke(user_input)
        return context_characters
    return None

def change_chapter_if_requested(response, chapter_num):
    """
    응답에서 '챕터 변경' 명령을 감지하고, 필요시 챕터를 변경하는 함수.
    """
    match = re.search(r"챕터\s*:\s*(\d+)\s*[으]?로 변경하겠습니다", response)

    if match:
        chapter_num = int(match.group(1) or match.group(2))  # 챕터 번호를 추출하여 정수로 변환
        return chapter_num, f"챕터 변경 완료 : {chapter_num}"
    else:
        print("챕터 변경 요청이 포함되어 있지 않습니다.")
        return chapter_num, None
    
def update_chatbot_output(memory, chatbot_output):
    """
    memory에서 최근 사용자와 어시스턴트 메시지를 가져와 chatbot_output에 추가하는 함수.
    """
    most_recent_assistant = memory.chat_memory.messages[-1].content  # 어시스턴트의 최근 메시지
    most_recent_user = memory.chat_memory.messages[-2].content  # 사용자의 최근 메시지
    chatbot_output.append((most_recent_user, most_recent_assistant))
    return chatbot_output

#### 3-1. 챗봇 파일 업로드

In [14]:
# 파일 업로드 처리 함수
# 이거 수정해서 벡터화 및 파일 db 생성해서 순서대로 부르게 하기
# gpt 보라
def chatbot_send_file(chat_history, file):
    global UPLOAD_TXT_PATH

    UPLOAD_TXT_PATH = save_uploaded_file(file)
    openai_db = text_vectorize(file)
    conversation_path = conversation_extraction(file)
    conversation_db =conversation_vectorize(conversation_path)

    # conversation_txt 를 임베딩하는 코드 추가

    chat_history.append(["사용자가 업로드한 파일입니다.",f"{file.name}"])
    return chat_history

#### 3-2. 챗봇 메세지 보내기 

In [15]:
chatbot_output = []

# 이벤트 처리 함수
def chatbot_send_message(cb_textbox):
    global context_books
    global chapter_num
    global memory
    global chatbot_output
    global context_characters
    # 사용자 입력 가져오기
    user_input = cb_textbox
    
    # 문맥 정보(책 정보) 검색
    context_books = retrieve_context_books(user_input, openai_db)

    # 문맥 정보(캐릭터 정보) 검색
    context_characters = retrieve_context_characters(user_input, conversation_db)

    # handle_user_input 함수를 통해 답변 생성
    response, _ = handle_user_input(user_input)
    #response, updated_history = CHATBOT.handle_user_input(user_input)
    
    print(response)
    
    # 음성 생성 함수 추가
    
    
    # 챕터 변경 로직

    chapter_num,new_input =change_chapter_if_requested(response, chapter_num)
    if new_input is not None:
        # 새로운 입력이 있는 경우에만 다시 handle_user_input을 호출
        response, _ = handle_user_input(new_input)
    '''
    match = re.search(r"챕터\s*:\s*(\d+)\s*[으]?로 변경하겠습니다", response)
    if match:
        chapter_num = int(match.group(1) or match.group(2))  # 챕터 번호를 추출하여 정수로 변환
        user_input = f"'챕터 변경 완료 : {chapter_num}'"
        response, updated_history = handle_user_input(user_input)
    else:
        print("챕터 변경 요청이 포함되어 있지 않습니다.")
    '''


    # 챗봇의 응답을 대화 기록에 추가

    chatbot_output = update_chatbot_output(memory, chatbot_output)
    '''
    most_recent_user = None
    most_recent_assistant = None

    most_recent_assistant = memory.chat_memory.messages[-1].content #이건 assistant이고
    most_recent_user = memory.chat_memory.messages[-2].content #이건 user네?
    chatbot_output.append((most_recent_user, most_recent_assistant))
    '''
    # 텍스트박스 초기화
    cb_textbox = ""
    print(chatbot_output)
    # 새로운 대화 기록과 초기화된 텍스트박스를 반환
    return chatbot_output, cb_textbox

#### 3-3. 챗 봇 음성 데이터 보내기

In [16]:
import whisper

# Whisper 모델 로드
audio_model = whisper.load_model("base")

def transcribe_audio(audio_file):
    if audio_file is None or audio_file== "":
        # 파일 경로가 없거나 비어있는 경우 아무 작업도 하지 않음
        print("음성 없음")
        return gr.update()

    
    # Whisper 모델을 사용하여 음성을 텍스트로 변환
    result = audio_model.transcribe(audio_file)
    recognized_text = result["text"]

    chatbot_output, cb_textbox = chatbot_send_message(recognized_text)
    
    return chatbot_output#,detected_language

In [17]:
import gradio as gr

# 프롬프트에 들어가야 하는 내용


with gr.Blocks() as app:
    with gr.Tab("음성 인식봇"):
        with gr.Column():
            gr.Markdown(
                value="""
<center><h1> 음성 인식 봇 </h1></center>
<center> AI 인공지능 봇입니다. 음성으로 묻거나 문서 요약, 일정 관리 등을 할 수 있습니다.</center>
"""
            )
            cb_chatbot = gr.Chatbot(
                value=[[None, "안녕하세요 AI 챗봇입니다. 무엇이든 질문하세요"],
                       ],
                label="구미챗봇",
            )
        with gr.Row():
            cb_textbox = gr.Textbox(
                lines=3,
                placeholder="입력창입니다.",
                container=False,
                scale=5
            ) # 채팅 입력

            audio_input = gr.Audio(
                sources= "microphone",
                type="filepath",
                label= "음성을 입력해주세요",
                scale=2
                ) # 음성 입력

            cb_send_btn = gr.Button(
                value="보내기",
                visible="primary",
                scale=1
            ) # 실행 버튼
            
            file_upload = gr.File(
                label="txt 파일을 업로드해주세요",
                type="filepath"
            ) # 파일 업로드
            
        with gr.Row():
            gr.Button(
                value="되돌리기"
            )
            gr.Button(
                value="초기화"
            )
        
        # 텍스트 입력 처리
        cb_send_btn.click(
            fn=chatbot_send_message,
            inputs=[cb_textbox],
            outputs=[cb_chatbot, cb_textbox]
        )
        
        # 파일 업로드 처리
        file_upload.change(
            fn= chatbot_send_file,
            inputs = [cb_chatbot,file_upload],#입력값 전달될 것 
            outputs=[cb_chatbot]#출력값 들어갈 곳
        )

        # 음성 입력 처리 
        audio_input.change(
            fn=transcribe_audio,
            inputs=[audio_input],  # 음성 입력 컴포넌트를 입력으로 사용
            outputs=[cb_chatbot],  # 텍스트 박스와 감지된 언어 출력
        )

    with gr.Tab("문서 요약봇"):
        pass
    with gr.Tab("일정 관리봇"):
        pass

app.launch(debug=True)

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


저장되었음
벡터 저장소에 저장된 문서 수: 14272
첫 번째 문서와 유사한 문서 검색 결과: [Document(page_content='CHAPTER I—M. MYRIEL\nIn 1815, M. Charles-François-Bienvenu Myriel was Bishop of D—— He was an old man of about seventy-five years of age; he had occupied the see of D—— since 1806.'), Document(page_content='CHAPTER I—M. MYRIEL\nIn 1815, M. Charles-François-Bienvenu Myriel was Bishop of D—— He was an old man of about seventy-five years of age; he had occupied the see of D—— since 1806.'), Document(page_content='CHAPTER I—M. MYRIEL\nIn 1815, M. Charles-François-Bienvenu Myriel was Bishop of D—— He was an old man of about seventy-five years of age; he had occupied the see of D—— since 1806.'), Document(page_content='CHAPTER I—M. MYRIEL\nIn 1815, M. Charles-François-Bienvenu Myriel was Bishop of D—— He was an old man of about seventy-five years of age; he had occupied the see of D—— since 1806.')]


c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Chapter 1:

Chapter 2:
"M. Myriel"의 대화문은 다음과 같습니다:

1. “Sire,” said M. Myriel, “you are looking at a good man, and I at a great man. Each of us can profit by it.”
Chapter 3:
"M. Myriel"의 대화문은 다음과 같습니다:

1. “Sire,” said M. Myriel, “you are looking at a good man, and I at a great man. Each of us can profit by it.”"M. Myriel"의 대화문은 다음과 같습니다:

1. “Monsieur the director of the hospital,” said he to him, “how many sick people have you at the present moment?”
2. “That was the number which I counted,” said the Bishop.
3. “That is what I observed.”
4. “So it seems to me.”
5. “That was what I said to myself.”
6. “That is the thought which occurred to me.”
7. “What would you have, Monseigneur?” said the director. “One must resign one’s self.”
8. “Monsieur,” said he, “how many beds do you think this hall alone would hold?”
9. “It would hold full twenty beds,” said he, as though speaking to himself. Then, raising his voice:—
10. “Hold, Monsieur the director of the hospital, I will tell you somethin

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 399, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 70, in __call__
    return await self.app(scope, receive, send)
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\starlette\applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\starlett

...

죄송합니다, 'Talim'에 대한 요청이 명확하지 않습니다. 더 자세히 설명해주시면 감사하겠습니다.
챕터 변경 요청이 포함되어 있지 않습니다.
[(' 전문단 읽어줘', "1815년에, M. Charles-François-Bienvenu Myriel은 D——의 주교였다. 그는 대략 75세의 노인이었고, 그는 1806년 이후로 D——의 자리를 차지했다.\n\n이 상세한 내용이 우리가 이야기하려는 실질적인 내용과 어떠한 연관성도 없지만, 모든 점에서의 정확성을 위해, 그가 교구에 도착한 순간부터 그에 대한 여러 소문과 논평을 여기에 언급하는 것은 불필요하지 않을 것이다. 참이든 거짓이든, 사람들에 대해 얘기되는 것은 그들의 삶에서, 그리고 그들의 운명에서, 그들이 하는 것만큼 중요한 자리를 차지한다. M. Myriel은 Aix의 의회의 한 원로의 아들이었다. 그래서 그는 대리인의 귀족에 속했다. 그의 아버지는 그를 자신의 직책의 후계자로 만들기 위해 그를 아주 어린 나이, 18세나 20세에 결혼시켰다고 했다. 이는 의회의 가족들 사이에 상당히 널리 퍼져 있는 관습에 따른 것이었다. 그러나 이 결혼에도 불구하고, Charles Myriel은 많은 이야기를 만들어냈다고 했다. 그는 잘 생겼지만 키가 조금 작았고, 우아하고, 기분 좋고, 지능적이었다. 그의 삶의 첫 부분 전체가 세상과 호의에 헌신되었다.\n\n혁명이 왔다. 사건들이 연속해서 벌어졌다. 의회의 가족들이 줄지어 쫓겨나고, 추적당하고, 사냥당하고, 흩어졌다. M. Charles Myriel은 혁명이 시작할 때 바로 이탈리아로 이주했다. 그곳에서 그의 아내는 오랫동안 고통 받았던 가슴의 병으로 죽었다. 그에게는 아이가 없었다. M. Myriel의 운명에서 다음에 무슨 일이 일어났을까? 옛날의 프랑스 사회의 붕괴, 그의 가족의 몰락, '93년의 비극적인 모습들은, 아마도 그것들을 멀리서 보는 이민자들에게 테러의 확대 힘으로 더욱 더 놀라운 것들이었을지도 모른다. 이것들이 그에게서 포기와 고독의 아이디어를 발아시키

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 399, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 70, in __call__
    return await self.app(scope, receive, send)
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\starlette\applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\starlett

혁명이 왔다. 사건들이 연속해서 벌어졌다. 의회의 가족들이 줄지어 쫓겨나고, 추적당하고, 사냥당하고, 흩어졌다. M. Charles Myriel은 혁명이 시작할 때 바로 이탈리아로 이주했다. 그곳에서 그의 아내는 오랫동안 고통 받았던 가슴의 병으로 죽었다. 그에게는 아이가 없었다. M. Myriel의 운명에서 다음에 무슨 일이 일어났을까? 옛날의 프랑스 사회의 붕괴, 그의 가족의 몰락, '93년의 비극적인 모습들은, 아마도 그것들을 멀리서 보는 이민자들에게 테러의 확대 힘으로 더욱 더 놀라운 것들이었을지도 모른다. 이것들이 그에게서 포기와 고독의 아이디어를 발아시키게 했을까? 그는 이러한 산만함, 그의 삶을 흡수하는 이러한 애정 속에서, 가끔 공공의 재앙들이 흔들리지 않는 사람을, 그의 존재와 재산에 공격하여 갑자기 신비하고 무서운 일격으로 충격을 받은 것일까? 아무도 말할 수 없었다. 알려진 것은, 그가 이탈리아로부터 돌아왔을 때 그는 사제였다는 것뿐이었다.
챕터 변경 요청이 포함되어 있지 않습니다.
[(' 전문단 읽어줘', "1815년에, M. Charles-François-Bienvenu Myriel은 D——의 주교였다. 그는 대략 75세의 노인이었고, 그는 1806년 이후로 D——의 자리를 차지했다.\n\n이 상세한 내용이 우리가 이야기하려는 실질적인 내용과 어떠한 연관성도 없지만, 모든 점에서의 정확성을 위해, 그가 교구에 도착한 순간부터 그에 대한 여러 소문과 논평을 여기에 언급하는 것은 불필요하지 않을 것이다. 참이든 거짓이든, 사람들에 대해 얘기되는 것은 그들의 삶에서, 그리고 그들의 운명에서, 그들이 하는 것만큼 중요한 자리를 차지한다. M. Myriel은 Aix의 의회의 한 원로의 아들이었다. 그래서 그는 대리인의 귀족에 속했다. 그의 아버지는 그를 자신의 직책의 후계자로 만들기 위해 그를 아주 어린 나이, 18세나 20세에 결혼시켰다고 했다. 이는 의회의 가족들 사이에 상당히 널리 퍼져 있는 관습에 따른 것이었다. 그러나 이 결혼에도 불구하

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 399, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 70, in __call__
    return await self.app(scope, receive, send)
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\starlette\applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\starlett

제 생각에 형명이란 사람이나 사건을 이 책에서 언급하지 않았습니다. 그러나 만약 '형명'이라는 사람이나 이벤트가 내 삶에 등장했다면, 그것이 나에게 어떤 영향을 미쳤는지에 따라 내 감정이 달라질 것입니다. 그를 만남으로 인해 기쁨, 놀람, 혹은 두려움과 같은 다양한 감정을 느낄 수 있을 것입니다. 그러나 이 책의 텍스트에는 그런 내용이 없으므로, 구체적인 감정을 설명하는 것은 어렵습니다.
챕터 변경 요청이 포함되어 있지 않습니다.
[(' 전문단 읽어줘', "1815년에, M. Charles-François-Bienvenu Myriel은 D——의 주교였다. 그는 대략 75세의 노인이었고, 그는 1806년 이후로 D——의 자리를 차지했다.\n\n이 상세한 내용이 우리가 이야기하려는 실질적인 내용과 어떠한 연관성도 없지만, 모든 점에서의 정확성을 위해, 그가 교구에 도착한 순간부터 그에 대한 여러 소문과 논평을 여기에 언급하는 것은 불필요하지 않을 것이다. 참이든 거짓이든, 사람들에 대해 얘기되는 것은 그들의 삶에서, 그리고 그들의 운명에서, 그들이 하는 것만큼 중요한 자리를 차지한다. M. Myriel은 Aix의 의회의 한 원로의 아들이었다. 그래서 그는 대리인의 귀족에 속했다. 그의 아버지는 그를 자신의 직책의 후계자로 만들기 위해 그를 아주 어린 나이, 18세나 20세에 결혼시켰다고 했다. 이는 의회의 가족들 사이에 상당히 널리 퍼져 있는 관습에 따른 것이었다. 그러나 이 결혼에도 불구하고, Charles Myriel은 많은 이야기를 만들어냈다고 했다. 그는 잘 생겼지만 키가 조금 작았고, 우아하고, 기분 좋고, 지능적이었다. 그의 삶의 첫 부분 전체가 세상과 호의에 헌신되었다.\n\n혁명이 왔다. 사건들이 연속해서 벌어졌다. 의회의 가족들이 줄지어 쫓겨나고, 추적당하고, 사냥당하고, 흩어졌다. M. Charles Myriel은 혁명이 시작할 때 바로 이탈리아로 이주했다. 그곳에서 그의 아내는 오랫동안 고통 받았던 가슴의 병으로 죽었다. 그에게는 아이가 없었다

In [ ]:
list(raw_book.keys())


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [ ]:
import gradio as gr
import shutil

def save_file(file):
    if file is not None:
        # file 은 임시 저장 결로를 나타내는 듯
        shutil.copy(file, "save_file.txt")

with gr.Blocks() as app:
    file_input = gr.File(label="업로드 파일")
    send_btn = gr.Button("파일 저장하기")

    send_btn.click(fn=save_file, inputs=file_input, outputs=None)

app.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [ ]:
file_path = f"txt/LESMISERABLES_CH1.txt"
system_message_for_conv_extraction = """
        너는 txt 파일에서 주인공을 찾고 대화문을 출력하는 어시스턴트야. 아래 지침에 따라 답변을 해줘야 해.
        1. 사용자가 '해당 챕터의 주인공 : '을 요청하면 '해당 챕터의 주인공 이름 : "챕터에 나오는 주인공 이름"'으로 답변해야 해.
        2. 사용자가 '"주인공 이름"의 모든 대화문'을 요청하면 해당 챕터의 "주인공 이름"의 모든 대화문을 출력해야 해.

        txt 파일은 다음과 같아:
        {txt_file}
    """

def conversation_emb():

    history_conv_extraction = []
    
    input_conv_extraction = '해당 챕터의 주인공 : '
    
    conversation_txt = ''

    chat_prompt_for_conv_extraction = ChatPromptTemplate.from_messages(
        [
            ("system", system_message_for_conv_extraction),
            MessagesPlaceholder(variable_name="history"), # 이전 대화 내용 저장용
            ("human", "{input}")
        ]
    )
    llm_chain_for_conv_extraction = LLMChain(
        llm=gpt_model,
        prompt=chat_prompt_for_conv_extraction,
    )

    main_character_response = llm_chain_for_conv_extraction.run({
        "input": input_conv_extraction,
        "history" : history_conv_extraction,
        "txt_file" : raw_book[2]
    })

    match = re.search(r'"(.*?)"', main_character_response)
    
    if match:  # 주인공 이름이 매칭된 경우
        main_character = match.group(1)
        input_conv_extraction = f'"{main_character}"의 모든 대화문'
        for chapter_number, dialogues in raw_book.items():
            print(f"Chapter {chapter_number}:")
            print(conversation_txt)

            conversation_txt += llm_chain_for_conv_extraction.run({
                "input": input_conv_extraction,
                "history": history_conv_extraction,
                "txt_file": dialogues[1]
            })
    with open('conversation.txt','w',encoding = 'utf-8') as file:
        file.write(conversation_txt)

    return conversation_txt

In [ ]:
raw_book[1]

['I—M. MYRIEL',
 'In 1815, M. Charles-François-Bienvenu Myriel was Bishop of D—— He was an old man of about seventy-five years of age; he had occupied the see of D—— since 1806.\n\nAlthough this detail has no connection whatever with the real substance of what we are about to relate, it will not be superfluous, if merely for the sake of exactness in all points, to mention here the various rumors and remarks which had been in circulation about him from the very moment when he arrived in the diocese. True or false, that which is said of men often occupies as important a place in their lives, and above all in their destinies, as that which they do. M. Myriel was the son of a councillor of the Parliament of Aix; hence he belonged to the nobility of the bar. It was said that his father, destining him to be the heir of his own post, had married him at a very early age, eighteen or twenty, in accordance with a custom which is rather widely prevalent in parliamentary families. In spite of this

In [ ]:
file_path = f"txt/LESMISERABLES_CH1.txt"
with open(file_path, 'r', encoding='utf-8') as f:
    txt_file_conv_extraction = f.read()

In [ ]:
history_conv_extraction = []

In [ ]:
input_conv_extraction = '해당 챕터의 주인공 : '

In [ ]:
system_message_for_conv_extraction = """
    너는 txt 파일에서 주인공을 찾고 대화문을 출력하는 어시스턴트야. 아래 지침에 따라 답변을 해줘야 해.
    1. 사용자가 '해당 챕터의 주인공 : '을 요청하면 '해당 챕터의 주인공 이름 : "챕터에 나오는 주인공 이름"'으로 답변해야 해.
    2. 사용자가 '"주인공 이름"의 모든 대화문'을 요청하면 해당 챕터의 "주인공 이름"의 모든 대화문을 출력해야 해.

    txt 파일은 다음과 같아:
    {txt_file}
"""


chat_prompt_for_conv_extraction = ChatPromptTemplate.from_messages(
    [
        ("system", system_message_for_conv_extraction),
        MessagesPlaceholder(variable_name="history"), # 이전 대화 내용 저장용
        ("human", "{input}")
    ]
)
llm_chain_for_conv_extraction = LLMChain(
    llm=gpt_model,
    prompt=chat_prompt_for_conv_extraction,
)

response = llm_chain_for_conv_extraction.run({
    "input": input_conv_extraction,
    "history" : history_conv_extraction,
    "txt_file" : raw_book[2]
})

In [ ]:
response

'해당 챕터의 주인공 이름 : "M. Myriel"'

In [ ]:
conversation_txt = ''
match = re.search(r'"(.*?)"', response)
main_character = match.group(1)
input_conv_extraction = f'"{main_character}"의 모든 대화문'

for chapter_number, dialogues in raw_book.items():
    print(f"Chapter {chapter_number}:")
    print(conversation_txt)

    if match:  # 주인공 이름이 매칭된 경우
        conversation_txt += llm_chain_for_conv_extraction.run({
            "input": input_conv_extraction,
            "history": history_conv_extraction,
            "txt_file": dialogues[1]
        })


Chapter 1:

Chapter 2:
"M. Myriel"의 대화문은 다음과 같습니다:

1. “Sire,” said M. Myriel, “you are looking at a good man, and I at a great man. Each of us can profit by it.”
Chapter 3:
"M. Myriel"의 대화문은 다음과 같습니다:

1. “Sire,” said M. Myriel, “you are looking at a good man, and I at a great man. Each of us can profit by it.”"M. Myriel"의 대화문은 다음과 같습니다:

1. “Monsieur the director of the hospital,” said he to him, “how many sick people have you at the present moment?”

2. “That was the number which I counted,” said the Bishop.

3. “That is what I observed.”

4. “So it seems to me.”

5. “That was what I said to myself.”

6. “That is the thought which occurred to me.”

7. “Monsieur,” said he, “how many beds do you think this hall alone would hold?”

8. “Hold, Monsieur the director of the hospital, I will tell you something. There is evidently a mistake here. There are thirty-six of you, in five or six small rooms. There are three of us here, and we have room for sixty. There is some mistake, I tell you;

In [ ]:
with open('conversation.txt','w',encoding = 'utf-8') as file:
    file.write(conversation_txt)

In [ ]:
raw_book.items()

dict_items([(1, ['I—M. MYRIEL', 'In 1815, M. Charles-François-Bienvenu Myriel was Bishop of D—— He was an old man of about seventy-five years of age; he had occupied the see of D—— since 1806.\n\nAlthough this detail has no connection whatever with the real substance of what we are about to relate, it will not be superfluous, if merely for the sake of exactness in all points, to mention here the various rumors and remarks which had been in circulation about him from the very moment when he arrived in the diocese. True or false, that which is said of men often occupies as important a place in their lives, and above all in their destinies, as that which they do. M. Myriel was the son of a councillor of the Parliament of Aix; hence he belonged to the nobility of the bar. It was said that his father, destining him to be the heir of his own post, had married him at a very early age, eighteen or twenty, in accordance with a custom which is rather widely prevalent in parliamentary families. I

In [ ]:
for chapter_number, dialogues in raw_book.items():
    print(f"Chapter {chapter_number}:")
    for dialogue in dialogues:
        print(dialogue)
    print("\n")  # 챕터 구분을 위해 줄바꿈 추가

Chapter 1:
I—M. MYRIEL
In 1815, M. Charles-François-Bienvenu Myriel was Bishop of D—— He was an old man of about seventy-five years of age; he had occupied the see of D—— since 1806.

Although this detail has no connection whatever with the real substance of what we are about to relate, it will not be superfluous, if merely for the sake of exactness in all points, to mention here the various rumors and remarks which had been in circulation about him from the very moment when he arrived in the diocese. True or false, that which is said of men often occupies as important a place in their lives, and above all in their destinies, as that which they do. M. Myriel was the son of a councillor of the Parliament of Aix; hence he belonged to the nobility of the bar. It was said that his father, destining him to be the heir of his own post, had married him at a very early age, eighteen or twenty, in accordance with a custom which is rather widely prevalent in parliamentary families. In spite of t

In [ ]:
conversation_txt

'"M. Myriel"의 대화문:\n1. “Monsieur the director of the hospital,” said he to him, “how many sick people have you at the present moment?”\n2. “That was the number which I counted,” said the Bishop.\n3. “That is what I observed.”\n4. “So it seems to me.”\n5. “That was what I said to myself.”\n6. “That is the thought which occurred to me.”\n7. “Monsieur,” said he, “how many beds do you think this hall alone would hold?”\n8. “Hold, Monsieur the director of the hospital, I will tell you something. There is evidently a mistake here. There are thirty-six of you, in five or six small rooms. There are three of us here, and we have room for sixty. There is some mistake, I tell you; you have my house, and I have yours. Give me back my house; you are at home here.”\n9. “And still I am quite cramped with it all!”\n10. “Hold!” cried the Bishop, “you are quite right, Madame Magloire.”\n\n"Mademoiselle Baptistine"의 대화문: 없음\n\n"Madame Magloire"의 대화문:\n1. “I should think so!” exclaimed Madame Magloire. “M

In [ ]:
raw_book

{1: ['I—M. MYRIEL',
  'In 1815, M. Charles-François-Bienvenu Myriel was Bishop of D—— He was an old man of about seventy-five years of age; he had occupied the see of D—— since 1806.\n\nAlthough this detail has no connection whatever with the real substance of what we are about to relate, it will not be superfluous, if merely for the sake of exactness in all points, to mention here the various rumors and remarks which had been in circulation about him from the very moment when he arrived in the diocese. True or false, that which is said of men often occupies as important a place in their lives, and above all in their destinies, as that which they do. M. Myriel was the son of a councillor of the Parliament of Aix; hence he belonged to the nobility of the bar. It was said that his father, destining him to be the heir of his own post, had married him at a very early age, eighteen or twenty, in accordance with a custom which is rather widely prevalent in parliamentary families. In spite of

In [ ]:
main_character

'M. MYRIEL'